In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    mean_squared_error, r2_score
)

In [6]:
def load_and_preprocess_cme_multifeature_from_flatdir(year: int, cme_data_dir: str) -> pd.DataFrame:
    cme_data_dir = Path(cme_data_dir)
    files = [cme_data_dir / f"univ{year}_{m:02d}.txt" for m in range(1, 13)]

    full_dates = pd.date_range(pd.Timestamp(year, 1, 1), pd.Timestamp(year, 12, 31), freq="D")

    dfs = []
    for f in files:
        if not f.exists():
            continue
        df = pd.read_fwf(f, skiprows=4, header=None)
        dfs.append(df)

    # 해당 연도 CME 파일이 하나도 없으면 0으로 채운 daily 리턴
    if not dfs:
        daily = pd.DataFrame({"date": full_dates})
        for c in [
            "cme_count","halo_count","partial_halo_count",
            "width_max","width_mean",
            "speed_linear_max","speed_linear_mean",
            "speed_init_max","speed_final_max","speed_20R_max",
            "accel_max","accel_min",
            "mass_sum","mass_max",
            "ke_sum","ke_max"
        ]:
            daily[c] = 0.0
        return daily

    cme = pd.concat(dfs, ignore_index=True)

    # 열이 부족하면 채워넣기(포맷 변형 방어)
    if cme.shape[1] < 13:
        for _ in range(13 - cme.shape[1]):
            cme[cme.shape[1]] = np.nan

    out = pd.DataFrame()
    out["date"] = pd.to_datetime(cme.iloc[:, 0], errors="coerce")

    central = cme.iloc[:, 2].astype(str)
    out["is_halo"] = central.str.contains("Halo", case=False, na=False).astype(int)
    out["is_partial_halo"] = cme.iloc[:, 12].astype(str).str.contains("Partial", case=False, na=False).astype(int)

    def num(col):
        return pd.to_numeric(
            cme.iloc[:, col].astype(str).str.replace("*", "", regex=False),
            errors="coerce"
        )

    out["width"] = num(3)
    out["speed_linear"] = num(4)
    out["speed_init"] = num(5)
    out["speed_final"] = num(6)
    out["speed_20R"] = num(7)
    out["accel"] = num(8)
    out["mass"] = num(9)
    out["kinetic_energy"] = num(10)

    out = out.dropna(subset=["date"])

    daily = (
        out.groupby("date")
        .agg(
            cme_count=("date", "count"),
            halo_count=("is_halo", "sum"),
            partial_halo_count=("is_partial_halo", "sum"),

            width_max=("width", "max"),
            width_mean=("width", "mean"),

            speed_linear_max=("speed_linear", "max"),
            speed_linear_mean=("speed_linear", "mean"),

            speed_init_max=("speed_init", "max"),
            speed_final_max=("speed_final", "max"),
            speed_20R_max=("speed_20R", "max"),

            accel_max=("accel", "max"),
            accel_min=("accel", "min"),

            mass_sum=("mass", "sum"),
            mass_max=("mass", "max"),

            ke_sum=("kinetic_energy", "sum"),
            ke_max=("kinetic_energy", "max"),
        )
        .reset_index()
    )

    # 연도 전체 날짜로 채우기
    daily = (
        daily.set_index("date")
        .reindex(full_dates)
        .rename_axis("date")
        .reset_index()
    )

    # NaN -> 0
    daily = daily.fillna(0.0)

    return daily

# -----------------------------
# hproton (DPD): 파서 에러/메타라인 방어 로더
# -----------------------------
def load_and_preprocess_hproton_from_dir(year: int, hproton_data_dir: str) -> pd.DataFrame:
    dpd_path = Path(hproton_data_dir) / f"{year}_DPD.txt"
    if not dpd_path.exists():
        raise FileNotFoundError(f"Missing DPD file: {dpd_path}")

    rows = []
    with open(dpd_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 6:
                continue
            try:
                y = int(float(parts[0])); m = int(float(parts[1])); d = int(float(parts[2]))
            except ValueError:
                continue

            vals = parts[:9] + [np.nan] * (9 - len(parts[:9]))
            rows.append([y, m, d] + vals[3:9])

    hp = pd.DataFrame(
        rows,
        columns=[
            "year","month","day",
            "p_gt_1MeV","p_gt_10MeV","p_gt_100MeV",
            "e_gt_0p6MeV","e_gt_2MeV",
            "neutron_pct"
        ]
    )

    hp["p_gt_100MeV"] = pd.to_numeric(hp["p_gt_100MeV"], errors="coerce").fillna(0.0).astype(float)
    hp["p_gt_100MeV"] = hp["p_gt_100MeV"].where(hp["p_gt_100MeV"] >= 0)
    hp["p_gt_100MeV"] = hp["p_gt_100MeV"].fillna(0.0)
    hp["date"] = pd.to_datetime(hp[["year","month","day"]])

    return hp[["date","p_gt_100MeV"]].sort_values("date").reset_index(drop=True)


# -----------------------------
# 여러 해를 합쳐서 연속 날짜축으로 align
# -----------------------------
def build_merged_all_years(start_year=1996, end_year=2018,
                           cme_dir="./cme_data", hproton_dir="./hproton_data"):
    cme_all = []
    hp_all = []

    for year in range(start_year, end_year + 1):
        # CME (해당 연도 파일이 없으면 skip)
        try:
            cme_daily = load_and_preprocess_cme_multifeature(year, cme_dir)
            cme_all.append(cme_daily)
        except ValueError:
            # pd.concat에 들어갈 파일이 없어서 생기는 경우 등
            continue

        # DPD (없으면 skip)
        dpd_path = f"{hproton_dir}/{year}_DPD.txt"
        try:
            hp_daily = load_and_preprocess_hproton(dpd_path)
            hp_all.append(hp_daily)
        except FileNotFoundError:
            continue

    cme_all = pd.concat(cme_all, ignore_index=True).sort_values("date").reset_index(drop=True)
    hp_all  = pd.concat(hp_all,  ignore_index=True).sort_values("date").reset_index(drop=True)

    # 전체 기간 공통 날짜로 맞추기
    start = max(cme_all["date"].min(), hp_all["date"].min())
    end   = min(cme_all["date"].max(), hp_all["date"].max())
    full_dates = pd.date_range(start, end, freq="D")

    cme_aligned = (
        cme_all.set_index("date")
        .reindex(full_dates, fill_value=0)
        .rename_axis("date")
        .reset_index()
    )
    hp_aligned = (
        hp_all.set_index("date")
        .reindex(full_dates, fill_value=0)
        .rename_axis("date")
        .reset_index()
    )

    merged = pd.merge(cme_aligned, hp_aligned, on="date", how="inner")
    return merged

# -----------------------------
# supervised dataset: lag/rolling + 미래 window max SEP 타깃
# -----------------------------
def make_supervised_dataset(
    df: pd.DataFrame,
    base_feature_cols=None,
    k_lag: int = 3,
    future_window: int = 3,
    use_log_target: bool = True
):
    out = df.sort_values("date").reset_index(drop=True).copy()

    if base_feature_cols is None:
        base_feature_cols = [c for c in out.columns if c not in ["date", "p_gt_100MeV"]]

    feat_cols = []
    for col in base_feature_cols:
        for lag in range(1, k_lag + 1):
            name = f"{col}_lag{lag}"
            out[name] = out[col].shift(lag)
            feat_cols.append(name)

        name = f"{col}_rollsum{k_lag}"
        out[name] = out[col].shift(1).rolling(k_lag).sum()
        feat_cols.append(name)

        name = f"{col}_rollmax{k_lag}"
        out[name] = out[col].shift(1).rolling(k_lag).max()
        feat_cols.append(name)

    # 미래 window max SEP
    out["target_sep"] = out["p_gt_100MeV"].shift(-1).rolling(future_window).max()

    model_df = out[["date"] + feat_cols + ["target_sep"]].dropna().reset_index(drop=True)

    X = model_df[feat_cols].values
    y = model_df["target_sep"].values

    if use_log_target:
        y = np.log10(y + 1.0)

    return model_df, feat_cols, X, y


# -----------------------------
# 실행: 1996~2018 데이터셋 생성
# -----------------------------
# if __name__ == "__main__":
#     merged_all = build_merged_all_years_flatdirs(
#         1996, 2018,
#         cme_data_dir="cme_data",
#         hproton_data_dir="hproton_data"
#     )

#     ds, feature_cols, X, y = make_supervised_dataset(
#         merged_all,
#         k_lag=3,
#         future_window=3,
#         use_log_target=True
#     )

#     print(len(merged_all), len(ds), (ds["target_sep"] > 0).sum())



In [7]:
# -----------------------------
# 1) CME 전처리: 월별 txt 모두 로드 -> Halo만 -> 일별 합/횟수 -> 누락일 0 채움
# -----------------------------

def load_and_preprocess_cme_multifeature(year: int, cme_dir: str):

    cme_dir = Path(cme_dir)
    files = [cme_dir / f"univ{year}_{m:02d}.txt" for m in range(1, 13)]

    dfs = []
    for f in files:
        if not f.exists():
            continue

        # fixed width
        df = pd.read_fwf(
            f,
            skiprows=4,
            header=None
        )
        dfs.append(df)

    cme = pd.concat(dfs, ignore_index=True)

    # -----------------------------------
    # 컬럼 구조 (실제 카탈로그 기준)
    # -----------------------------------
    # 0 : Date
    # 1 : Time
    # 2 : Central PA   (or Halo)
    # 3 : Width
    # 4 : Linear speed
    # 5 : 2nd order speed (initial)
    # 6 : 2nd order speed (final)
    # 7 : 2nd order speed (20R)
    # 8 : Accel
    # 9 : Mass
    # 10: Kinetic energy
    # 11: MPA
    # 12: Remarks

    out = pd.DataFrame()

    out["date"] = pd.to_datetime(cme.iloc[:, 0], errors="coerce")

    # halo / partial halo flag
    central = cme.iloc[:, 2].astype(str)
    out["is_halo"] = central.str.contains("Halo", case=False, na=False).astype(int)
    out["is_partial_halo"] = cme.iloc[:, 12].astype(str).str.contains(
        "Partial", case=False, na=False
    ).astype(int)

    # numeric columns
    def num(col):
        return pd.to_numeric(
            cme.iloc[:, col].astype(str).str.replace("*", "", regex=False),
            errors="coerce"
        )

    out["width"] = num(3)
    out["speed_linear"] = num(4)
    out["speed_init"] = num(5)
    out["speed_final"] = num(6)
    out["speed_20R"] = num(7)
    out["accel"] = num(8)

    # mass, energy는 ------- 같은 값이 있으므로 그대로 numeric 처리
    out["mass"] = num(9)
    out["kinetic_energy"] = num(10)

    out = out.dropna(subset=["date"])

    # -----------------------------------
    # 하루 단위 aggregation
    # -----------------------------------
    daily = (
        out
        .groupby("date")
        .agg(
            cme_count = ("date", "count"),

            halo_count = ("is_halo", "sum"),
            partial_halo_count = ("is_partial_halo", "sum"),

            width_max = ("width", "max"),
            width_mean = ("width", "mean"),

            speed_linear_max = ("speed_linear", "max"),
            speed_linear_mean = ("speed_linear", "mean"),

            speed_init_max = ("speed_init", "max"),
            speed_final_max = ("speed_final", "max"),
            speed_20R_max = ("speed_20R", "max"),

            accel_max = ("accel", "max"),
            accel_min = ("accel", "min"),

            mass_sum = ("mass", "sum"),
            mass_max = ("mass", "max"),

            ke_sum = ("kinetic_energy", "sum"),
            ke_max = ("kinetic_energy", "max"),
        )
        .reset_index()
    )

    # -----------------------------------
    # 날짜 채우기
    # -----------------------------------
    full_dates = pd.date_range(
        pd.Timestamp(year, 1, 1),
        pd.Timestamp(year, 12, 31),
        freq="D"
    )

    daily = (
        daily
        .set_index("date")
        .reindex(full_dates)
        .rename_axis("date")
        .reset_index()
    )

    # count 계열은 0, 나머지는 0으로 채워줌(모델용)
    count_cols = [
        "cme_count", "halo_count", "partial_halo_count"
    ]
    for c in count_cols:
        daily[c] = daily[c].fillna(0)

    other_cols = [c for c in daily.columns if c not in ["date"] + count_cols]
    daily[other_cols] = daily[other_cols].fillna(0)

    return daily



# -----------------------------
# 2) hproton 전처리: 메타라인 섞여도 안전하게 -> date 만들기 -> p_gt_100만
# -----------------------------

def load_and_preprocess_hproton(dpd_path: str) -> pd.DataFrame:
    rows = []
    with open(dpd_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            parts = line.split()
            if len(parts) < 6:   # 최소 year month day p1 p10 p100 있어야 함
                continue

            # 첫 3개가 정수여야 데이터 줄로 인정
            try:
                y = int(float(parts[0]))
                m = int(float(parts[1]))
                d = int(float(parts[2]))
            except ValueError:
                continue

            # 데이터 줄이면 앞 9개까지 안전하게 읽기 (없으면 NaN 채움)
            vals = parts[:9] + [np.nan] * (9 - len(parts[:9]))
            rows.append([y, m, d] + vals[3:9])

    hp = pd.DataFrame(
        rows,
        columns=[
            "year", "month", "day",
            "p_gt_1MeV", "p_gt_10MeV", "p_gt_100MeV",
            "e_gt_0p6MeV", "e_gt_2MeV",
            "neutron_pct"
        ]
    )

    # 숫자 변환
    for c in ["p_gt_1MeV","p_gt_10MeV","p_gt_100MeV","e_gt_0p6MeV","e_gt_2MeV","neutron_pct"]:
        hp[c] = pd.to_numeric(hp[c], errors="coerce")

    hp["date"] = pd.to_datetime(hp[["year","month","day"]])
    hp = hp[["date", "p_gt_100MeV"]].sort_values("date").reset_index(drop=True)

    # 결측은 0으로(원하면 그대로 NaN 둬도 됨)
    hp["p_gt_100MeV"] = hp["p_gt_100MeV"].fillna(0.0)

    return hp



# -----------------------------
# 3) CME + hproton 길이 n 맞추기(동일 date index로 reindex)
# -----------------------------
def align_daily_series(cme_daily: pd.DataFrame, hp_daily: pd.DataFrame) -> pd.DataFrame:
    # 공통 날짜 범위(교집합)로 맞추고 싶으면 여기서 start/end 조절 가능
    start = max(cme_daily["date"].min(), hp_daily["date"].min())
    end   = min(cme_daily["date"].max(), hp_daily["date"].max())
    full_dates = pd.date_range(start, end, freq="D")

    cme_aligned = (
        cme_daily.set_index("date")
        .reindex(full_dates, fill_value=0)
        .rename_axis("date")
        .reset_index()
    )
    hp_aligned = (
        hp_daily.set_index("date")
        .reindex(full_dates, fill_value=0)
        .rename_axis("date")
        .reset_index()
    )

    merged = pd.merge(cme_aligned, hp_aligned, on="date", how="inner")
    return merged


# -----------------------------
# 4) (과거 k일 CME) -> (오늘 SEP) 학습용 테이블 생성
# -----------------------------
def make_supervised_dataset(
    df: pd.DataFrame,
    base_feature_cols=None,
    k_lag: int = 3,
    future_window: int = 3,
    use_log_target: bool = True
):
    """
    df: must include columns ['date', 'p_gt_100MeV'] + CME feature columns

    base_feature_cols: CME feature columns list.
      - None이면 자동으로 (date, p_gt_100MeV) 제외한 모든 컬럼을 feature로 사용.
    """

    out = df.sort_values("date").reset_index(drop=True).copy()

    # ✅ feature 컬럼 자동 선택
    if base_feature_cols is None:
        base_feature_cols = [c for c in out.columns if c not in ["date", "p_gt_100MeV"]]

    # ------------------------
    # 1) lag / rolling features
    # ------------------------
    feat_cols = []
    for col in base_feature_cols:
        # lag
        for lag in range(1, k_lag + 1):
            name = f"{col}_lag{lag}"
            out[name] = out[col].shift(lag)
            feat_cols.append(name)

        # rolling sum (최근 k일 합)
        name = f"{col}_rollsum{k_lag}"
        out[name] = out[col].shift(1).rolling(k_lag).sum()
        feat_cols.append(name)

        # rolling max (최근 k일 최대)
        name = f"{col}_rollmax{k_lag}"
        out[name] = out[col].shift(1).rolling(k_lag).max()
        feat_cols.append(name)

    # ------------------------
    # 2) target: 미래 window max SEP
    # ------------------------
    out["target_sep"] = (
        out["p_gt_100MeV"]
        .shift(-1)
        .rolling(future_window)
        .max()
    )

    # ------------------------
    # 3) dropna and return
    # ------------------------
    model_df = out[["date"] + feat_cols + ["target_sep"]].dropna().reset_index(drop=True)

    X = model_df[feat_cols].values
    y = model_df["target_sep"].values

    if use_log_target:
        y = np.log10(y + 1.0)

    return model_df, feat_cols, X, y




# -----------------------------
# 5) 모델 학습/평가 (시간 순서 유지)
# -----------------------------




def train_and_evaluate_two_stage(
    ds: pd.DataFrame,
    feature_cols: list,
    split_ratio: float = 0.8,
    threshold: float = 0.0,      # ✅ event 정의: target_sep > threshold
    use_log_intensity: bool = True,
    random_state: int = 0,
):
    """
    ds: make_supervised_dataset()가 반환한 model_df (date, feature_cols..., target_sep 포함)
    feature_cols: feature column names
    threshold: event 기준(>100MeV pfu를 어떤 값 이상으로 event로 볼지)
               - 너가 '0보다 크면 발생'으로 갈 거면 0.0
               - NOAA식 임계값을 쓰고 싶으면 예: 1.0 또는 10.0 등으로 바꿔도 됨
    """

    # -------------------------
    # 0) Train/Test time split
    # -------------------------
    ds = ds.sort_values("date").reset_index(drop=True)
    n = len(ds)
    split = int(n * split_ratio)

    train_df = ds.iloc[:split].copy()
    test_df  = ds.iloc[split:].copy()

    X_train = train_df[feature_cols].values
    X_test  = test_df[feature_cols].values

    # -------------------------
    # 1) Stage-1: Event classifier
    # -------------------------
    y_train_cls = (train_df["target_sep"].values > threshold).astype(int)
    y_test_cls  = (test_df["target_sep"].values > threshold).astype(int)

    clf = RandomForestClassifier(
        n_estimators=500,
        max_depth=8,
        random_state=random_state,
        n_jobs=-1,
        class_weight="balanced"
    )
    clf.fit(X_train, y_train_cls)

    p_test = clf.predict_proba(X_test)[:, 1]

    # 분류 성능(희소 이벤트면 PR-AUC가 더 중요)
    auc = roc_auc_score(y_test_cls, p_test) if len(np.unique(y_test_cls)) > 1 else np.nan
    pr_auc = average_precision_score(y_test_cls, p_test) if len(np.unique(y_test_cls)) > 1 else np.nan

    # -------------------------
    # 2) Stage-2: Intensity regressor (event인 샘플만)
    # -------------------------
    train_event = train_df[train_df["target_sep"] > threshold].copy()
    test_event  = test_df[test_df["target_sep"] > threshold].copy()

    # event가 너무 적으면 회귀 자체가 불안정할 수 있음
    if len(train_event) < 10:
        raise ValueError(f"Too few event samples in train set: {len(train_event)}. "
                         f"Try lowering threshold or using more years.")

    X_train_reg = train_event[feature_cols].values
    X_test_reg  = test_event[feature_cols].values

    y_train_reg_raw = train_event["target_sep"].values
    y_test_reg_raw  = test_event["target_sep"].values

    if use_log_intensity:
        y_train_reg = np.log10(y_train_reg_raw + 1.0)
        y_test_reg  = np.log10(y_test_reg_raw + 1.0)
    else:
        y_train_reg = y_train_reg_raw
        y_test_reg  = y_test_reg_raw

    reg = RandomForestRegressor(
        n_estimators=800,
        max_depth=10,
        random_state=random_state,
        n_jobs=-1
    )
    reg.fit(X_train_reg, y_train_reg)

    # test의 "진짜 event"들에 대해서만 intensity 회귀 평가
    pred_reg = reg.predict(X_test_reg)

    mse = mean_squared_error(y_test_reg, pred_reg)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_reg, pred_reg) if len(y_test_reg) >= 2 else np.nan

    # -------------------------
    # 3) Two-stage 합성 예측(원하면 활용)
    # -------------------------
    # test 전체 날짜에 대해:
    #   - event 확률 p_test
    #   - intensity는 "예측 event"에만 계산해도 되지만,
    #     기대값 형태로 쓰려면 전체에 대해 reg를 돌려도 됨
    pred_reg_all = reg.predict(X_test)  # log 스케일일 수도 있음

    if use_log_intensity:
        intensity_all = np.maximum(0.0, 10**pred_reg_all - 1.0)
    else:
        intensity_all = np.maximum(0.0, pred_reg_all)

    expected_sep = p_test * intensity_all

    results = {
        "classifier_auc": auc,
        "classifier_pr_auc": pr_auc,
        "regressor_rmse": rmse,    # (log 스케일이면 log 기준 RMSE)
        "regressor_r2": r2,
        "n_train": len(train_df),
        "n_test": len(test_df),
        "n_train_event": len(train_event),
        "n_test_event": len(test_event),
    }

    # test 결과 dataframe(원하면 저장/플롯)
    pred_df = pd.DataFrame({
        "date": test_df["date"].values,
        "target_sep": test_df["target_sep"].values,
        "event_true": y_test_cls,
        "p_event": p_test,
        "intensity_pred": intensity_all,
        "expected_sep": expected_sep
    })

    return clf, reg, results, pred_df

# -----------------------------
# 실행 예시
# -----------------------------
if __name__ == "__main__":
    merged = build_merged_all_years(
        start_year=1996,
        end_year=2018,
        cme_dir="./cme_data",
        hproton_dir="./hproton_data"
    )

    ds, feature_cols, X, y = make_supervised_dataset(
        merged,
        k_lag=3,
        future_window=3,
        use_log_target=True
    )

    thr = ds["target_sep"].quantile(0.995)
    print("auto threshold:", thr)
    print("event rate:", (ds["target_sep"] > thr).mean())


    # ✅ 2-stage 실행
    clf, reg, results, pred_df = train_and_evaluate_two_stage(
        ds,
        feature_cols,
        threshold=thr
    )

    print("Results:", results)
    print("Dataset length:", len(ds))
    print("Event days:", (ds["target_sep"] > thr).sum())


C:\Users\User\AppData\Local\Temp\ipykernel_24424\3687323273.py:279: RuntimeWarning: invalid value encountered in log10
  y = np.log10(y + 1.0)


auto threshold: 1185000.0000000363
event rate: 0.005105217283028265
Results: {'classifier_auc': np.float64(0.9912718204488778), 'classifier_pr_auc': np.float64(0.46883468834688347), 'regressor_rmse': np.float64(0.11695562440151919), 'regressor_r2': 0.0, 'n_train': 6424, 'n_test': 1607, 'n_train_event': 38, 'n_test_event': 3}
Dataset length: 8031
Event days: 41


In [16]:
s = ds["target_sep"]

print("target_sep > 0 :", (s > 0).mean())
print("quantiles:", s.quantile([0, 0.5, 0.9, 0.95, 0.99, 0.999]))
print("top10:", s.sort_values(ascending=False).head(10).to_list())


target_sep > 0 : 0.9905366704021915
quantiles: 0.000   -1.000000e+05
0.500    3.500000e+03
0.900    4.700000e+03
0.950    6.500000e+03
0.990    2.800000e+05
0.999    6.073000e+06
Name: target_sep, dtype: float64
top10: [13000000.0, 13000000.0, 13000000.0, 11000000.0, 11000000.0, 11000000.0, 6100000.0, 6100000.0, 6100000.0, 5200000.0]


In [18]:
def summarize_event_rates(ds):
    for thr in [0, 1e-4, 1e-3, 1e-2, 5e-2, 0.1, 0.5, 1.0, 5.0, 10.0]:
        rate = (ds["target_sep"] > thr).mean()
        print(f"thr={thr:>7}  event_rate={rate:.4f}  events={int((ds['target_sep']>thr).sum())}")

summarize_event_rates(ds)


thr=      0  event_rate=0.9905  events=7955
thr= 0.0001  event_rate=0.9905  events=7955
thr=  0.001  event_rate=0.9905  events=7955
thr=   0.01  event_rate=0.9905  events=7955
thr=   0.05  event_rate=0.9905  events=7955
thr=    0.1  event_rate=0.9905  events=7955
thr=    0.5  event_rate=0.9905  events=7955
thr=    1.0  event_rate=0.9905  events=7955
thr=    5.0  event_rate=0.9905  events=7955
thr=   10.0  event_rate=0.9905  events=7955
